# **1. Data Collection**

In [1]:
# Imports
import praw
import pandas as pd
import numpy as np

### Initialize PRAW API

In [2]:
# Initialize PRAW
reddit = praw.Reddit(
    client_id='qSrQqSrnV9EgIimmHRjj7Q',
    client_secret='W9uIYSMSrekNc77Riw-IUfAFAFebAA',
    user_agent='myredditapp:v1.2.3 by u/Independent_Math_881',
    username='Independent_Math_881',
    password='DSI911!123'
)

### __WSB__: Scrape posts from the *WallStreetBets* subreddit, across each new, controversial, hot, rising and top posts, to keep within permitted rate limitations.  
#### Note, this was done on 10/19/23 and included 2,610 unique posts. 

In [19]:
# Write function to pull and aggregate subreddit data posts, remove duplicates and convert to dataframe 
def post_aggregator(subreddit_name):
    x_subreddit = reddit.subreddit(subreddit_name)
    new_posts = x_subreddit.new(limit=999)
    controversial_posts = x_subreddit.controversial(limit=999)
    hot_posts = x_subreddit.hot(limit=999)
    rising_posts = x_subreddit.rising(limit=999)
    top_posts = x_subreddit.top(limit=999)
    
    data_agg = []

    # Aggregate posts
    for post in new_posts:
        data_agg.append([post.created_utc, post.title, post.selftext, post.subreddit])
    for post in controversial_posts:
        data_agg.append([post.created_utc, post.title, post.selftext, post.subreddit])
    for post in hot_posts:
        data_agg.append([post.created_utc, post.title, post.selftext, post.subreddit])
    for post in rising_posts:
        data_agg.append([post.created_utc, post.title, post.selftext, post.subreddit])
    for post in top_posts:
        data_agg.append([post.created_utc, post.title, post.selftext, post.subreddit])

    # Turn aggregated data into a dataframe
    data_agg = pd.DataFrame(data_agg, columns = ['created_utc', 'title', 'self_text', 'subreddit'])
    
    # Drop duplicates
    data_agg = data_agg[~data_agg.duplicated(['created_utc', 'title', 'self_text', 'subreddit'])]
    
    # Add new column for total text (title + body)
    data_agg['text'] = data_agg['title'] + ' ' + data_agg['self_text']
    
    return data_agg

In [20]:
# Run function on WallStreetBets data
wsb = post_aggregator('wallstreetbets')

In [21]:
# Preview of df and size
print(wsb.shape)
wsb.head()

(2610, 5)


,created_utc,title,self_text,subreddit,text
0,1.697760e+09,Most Anticipated Earnings Releases for the wee...,,wallstreetbets,Most Anticipated Earnings Releases for the wee...
1,1.697759e+09,Robinhood Assignment. What are my options ?,\n So I had some credit spreads on spy expirin...,wallstreetbets,Robinhood Assignment. What are my options ? \n...
2,1.697758e+09,Calls on $TSLA?,,wallstreetbets,Calls on $TSLA?
3,1.697757e+09,Can we get a gamma $BIG,,wallstreetbets,Can we get a gamma $BIG
4,1.697757e+09,ServiceNow Earnings,What are everyone's feelings on servicenow ear...,wallstreetbets,ServiceNow Earnings What are everyone's feelin...


In [22]:
# Save data as .csv
wsb.to_csv('data/wsb_raw.csv', index=False)

### __CMS__: Scrape posts from the *CryptoMoonShots* subreddit, across each new, controversial, hot, rising and top posts, to keep within permitted rate limitations.  
#### Note, this was done on 10/19/23 and included 2,661 unique posts. 

In [23]:
# Run function on CryptoMoonShots data
cms = post_aggregator('CryptoMoonShots')

In [24]:
# Preview of df and size
print(cms.shape)
cms.head()

(2661, 5)


,created_utc,title,self_text,subreddit,text
0,1.697750e+09,PonyShibaInu - an adorable unique hybrid of Sh...,Pony Shiba Inu is the hottest new meme coin in...,CryptoMoonShots,PonyShibaInu - an adorable unique hybrid of Sh...
1,1.697749e+09,While you hunt meme and trashcoins Amazon rele...,I recommend to read more on either https://euc...,CryptoMoonShots,While you hunt meme and trashcoins Amazon rele...
2,1.697746e+09,Could $CUMINU $8m mc beat OnlyFans valued at $...,Cummuniti the new 18+ platform will change the...,CryptoMoonShots,Could $CUMINU $8m mc beat OnlyFans valued at $...
3,1.697745e+09,The Revolutionary Arsenal 2.0: Elevating Your ...,"\nGamers and blockchain enthusiasts, hold on...",CryptoMoonShots,The Revolutionary Arsenal 2.0: Elevating Your ...
4,1.697740e+09,Qtum Strikes the Perfect Balance: The Blockcha...,The landscape of blockchain technology is cons...,CryptoMoonShots,Qtum Strikes the Perfect Balance: The Blockcha...


In [25]:
# Save data as .csv
cms.to_csv('data/cms_raw.csv', index=False)

### Combine two dataframes into one for main analysis

In [35]:
# Concatenate wsb and cms into df
df = pd.concat([cms, wsb])
df.head()

,created_utc,title,self_text,subreddit,text
0,1.697750e+09,PonyShibaInu - an adorable unique hybrid of Sh...,Pony Shiba Inu is the hottest new meme coin in...,CryptoMoonShots,PonyShibaInu - an adorable unique hybrid of Sh...
1,1.697749e+09,While you hunt meme and trashcoins Amazon rele...,I recommend to read more on either https://euc...,CryptoMoonShots,While you hunt meme and trashcoins Amazon rele...
2,1.697746e+09,Could $CUMINU $8m mc beat OnlyFans valued at $...,Cummuniti the new 18+ platform will change the...,CryptoMoonShots,Could $CUMINU $8m mc beat OnlyFans valued at $...
3,1.697745e+09,The Revolutionary Arsenal 2.0: Elevating Your ...,"\nGamers and blockchain enthusiasts, hold on...",CryptoMoonShots,The Revolutionary Arsenal 2.0: Elevating Your ...
4,1.697740e+09,Qtum Strikes the Perfect Balance: The Blockcha...,The landscape of blockchain technology is cons...,CryptoMoonShots,Qtum Strikes the Perfect Balance: The Blockcha...


In [36]:
# Binarize 'subreddit' column
df['subreddit'] = np.where(df['subreddit'] == 'wallstreetbets', 0, 1)

df.head()

,created_utc,title,self_text,subreddit,text
0,1.697750e+09,PonyShibaInu - an adorable unique hybrid of Sh...,Pony Shiba Inu is the hottest new meme coin in...,1,PonyShibaInu - an adorable unique hybrid of Sh...
1,1.697749e+09,While you hunt meme and trashcoins Amazon rele...,I recommend to read more on either https://euc...,1,While you hunt meme and trashcoins Amazon rele...
2,1.697746e+09,Could $CUMINU $8m mc beat OnlyFans valued at $...,Cummuniti the new 18+ platform will change the...,1,Could $CUMINU $8m mc beat OnlyFans valued at $...
3,1.697745e+09,The Revolutionary Arsenal 2.0: Elevating Your ...,"\nGamers and blockchain enthusiasts, hold on...",1,The Revolutionary Arsenal 2.0: Elevating Your ...
4,1.697740e+09,Qtum Strikes the Perfect Balance: The Blockcha...,The landscape of blockchain technology is cons...,1,Qtum Strikes the Perfect Balance: The Blockcha...


In [38]:
# Save down .csv file
df.to_csv('data/agg_data_wsb_cms.csv', index = False)